In [5]:
# import package
from datetime import datetime as dtime
from datetime import date,timedelta
from urllib.request import urlopen
import requests
from dateutil import rrule
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import json
import time
import os

In [ ]:
DataPath = "./Data/"

In [ ]:
# 爬取每月股價的目標網站並包裝成函式
def craw_one_month(stock_number,date):
    url = (
        "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+
        date.strftime('%Y%m%d')+
        "&stockNo="+
        str(stock_number)
    )
    data = json.loads(urlopen(url).read())
    return pd.DataFrame(data['data'],columns=data['fields'])

In [ ]:
# 根據使用者輸入的日期，以月為單位，重複呼叫爬取月股價的函式
def craw_stock(stock_number, start_month):
    b_month = date(*[int(x) for x in start_month.split('-')])
    now = dtime.now().strftime("%Y-%m-%d")         # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    
    result = pd.DataFrame()
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        result = pd.concat([result,craw_one_month(stock_number,dt)],ignore_index=True)
        time.sleep(2000.0/1000.0);
    
    return result

In [ ]:
# 建立資料夾
def MKDir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)

In [ ]:
# 建立資料夾，搜尋並且存股票資料
def rrule_test(stock_number, start_month):
    MKDir(DataPath+str(stock_number))
    b_month = date(*[int(x) for x in start_month.split('-')])
    print(b_month.strftime('%Y-%m'))
    now = dtime.now().strftime("%Y-%m-%d")         # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    print(b_month)
    print(e_month)
    
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        print(dt)
        CurrentData = DataPath + str(stock_number) + '/' + dt.strftime('%Y-%m') + '.csv'
        print(CurrentData)
        if os.path.isfile(CurrentData):
            print("檔案存在。")
        else:
            print("檔案不存在。")
            result = pd.DataFrame()
            result = pd.concat([result,craw_one_month(stock_number,dt)],ignore_index=True)
            result.to_csv(CurrentData, encoding='utf_8_sig')
            time.sleep(2000.0/1000.0)
        
        


In [10]:
# 取得證券編號
def get_stock_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    print(df)
    

In [13]:
def create_tw_stock_info_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
    df = df.dropna(how='any')
    df = df.reset_index(drop=True)
    new_df = df['有價證券代號及名稱'].str.replace(u'\u3000',' ').str.split(u' ',expand=True)
    new_df.columns = ['Ticker', 'StockName']
    new_df['Sector'] = df['產業別']
    return new_df

In [ ]:
rrule_test(3034, "2020-05-1")

In [ ]:
df = craw_stock(3034,"2020-10-1")

In [ ]:
df.to_csv('./Data/3034.csv', encoding='utf_8_sig')

In [ ]:
MKDir('Data')

In [15]:
list = create_tw_stock_info_list()
print(list)

ImportError: lxml not found, please install it